# Análise Exploratória de Dados: Envelhecimento Ativo no Brasil

## O que é este notebook?

Este documento apresenta uma análise completa sobre o **envelhecimento ativo** no Brasil, utilizando dados da Pesquisa Nacional de Saúde (PNS 2019). Aqui, transformamos dados complexos em insights práticos para orientar políticas públicas e ações sociais.

### Por que isso importa?

O Brasil está envelhecendo rapidamente. Em 2019, havia mais de 30 milhões de pessoas com 60 anos ou mais. Garantir que esses idosos tenham uma vida digna, saudável e participativa é um desafio nacional que exige dados precisos e ações direcionadas.

### O que vamos descobrir?

Ao longo desta análise, responderemos perguntas como:
- Como está a qualidade de vida dos idosos brasileiros?
- Quais municípios mais precisam de atenção?
- Quais fatores influenciam o envelhecimento saudável?
- Existem grupos diferentes de idosos com necessidades distintas?

### Como funciona o Índice de Envelhecimento Ativo (AAI)?

Criamos um "termômetro" chamado AAI que mede o envelhecimento ativo combinando quatro pilares principais:
1. **Saúde**: Capacidade funcional e controle de doenças crônicas
2. **Participação Social**: Engajamento comunitário e acesso a redes de apoio
3. **Segurança Econômica**: Renda adequada e proteção social
4. **Educação e Acesso**: Conhecimento e conectividade digital

Cada pilar recebe uma nota de 0 a 1, e o AAI final é a média desses pilares.

### Nossa abordagem rigorosa

Usamos técnicas estatísticas avançadas para garantir que os resultados representem toda a população idosa brasileira, não apenas as pessoas entrevistadas. Isso inclui:
- **Pesos amostrais**: Para corrigir diferenças na probabilidade de seleção
- **Intervalos de confiança**: Para mostrar a precisão das nossas estimativas
- **Bootstrap**: Para validar a robustez dos resultados

---

*Este notebook foi desenvolvido para ser acessível a gestores públicos, profissionais de saúde, pesquisadores e qualquer pessoa interessada em melhorar a qualidade de vida dos idosos brasileiros.*

In [1]:
import pandas as pd

df = pd.read_csv("data/processed/pns_2019_pandas.csv")    
df.head(5)

,uf,estrato,upa,id_domicilio,num_pessoas_domicilio,area_metropolitana,mora_sozinho,sexo,idade,raca_cor,...,iadl_score,functional_raw,functional_score,dependencia_SUS,cobertura_influenza,autoav_z,multimorb_z,functional_z,health_score_raw,health_score
0,Rondônia,1110011,110000016,1,1,1,Cônjuge ou companheiro(a),Masculino,69.0,Parda,...,7,18,0.10,0,0,NaN,-1.027069,0.187726,NaN,NaN
1,Rondônia,1110011,110000016,1,1,1,Outro morador,Feminino,78.0,Parda,...,5,17,0.15,0,0,NaN,-1.027069,0.522010,NaN,NaN
2,Rondônia,1110011,110000016,1,1,1,Pessoa de referência,Masculino,81.0,Parda,...,2,9,0.55,0,2,-0.344963,1.200084,3.196276,3.167954,0.744213
3,Rondônia,1110011,110000016,1,1,1,Pessoa de referência,Masculino,81.0,Parda,...,7,19,0.05,0,2,-0.344963,0.795147,-0.146557,-0.559990,0.305083
4,Rondônia,1110011,110000016,1,1,1,Pessoa de referência,Feminino,72.0,Parda,...,7,16,0.20,0,0,NaN,-1.027069,0.856293,NaN,NaN


# Análise Avançada do Índice de Envelhecimento Ativo (AAI) - PNS 2019

Este notebook realiza uma análise completa dos dados da Pesquisa Nacional de Saúde (PNS 2019) para construir e validar o Índice de Envelhecimento Ativo (AAI) no Brasil. 

**Objetivos principais:**
- Construir um índice multidimensional de envelhecimento ativo
- Identificar municípios prioritários para intervenções
- Analisar desigualdades por subgrupos populacionais
- Desenvolver modelos preditivos de vulnerabilidade

**Metodologia:**
- Análise survey-aware com pesos amostrais
- Intervalos de confiança via bootstrap
- Agregação municipal com controle de qualidade
- Modelagem preditiva com validação cruzada

**Outputs esperados:**
- Scores municipais com intervalos de confiança
- Lista de municípios prioritários
- Perfis de envelhecimento via clustering
- Drivers de vulnerabilidade identificados
- Policy brief automatizado
- Visualizações e datasets processados

In [2]:
print("Colunas disponíveis no df:")
print(df.columns.tolist())

Colunas disponíveis no df:
['uf', 'estrato', 'upa', 'id_domicilio', 'num_pessoas_domicilio', 'area_metropolitana', 'mora_sozinho', 'sexo', 'idade', 'raca_cor', 'situacao_ocupacional', 'anos_estudo', 'renda_percapita', 'possui_plano_saude', 'consulta_12m', 'internacoes_12m', 'atendimento_sus', 'dificuldade_alimentar', 'dificuldade_banho', 'dificuldade_vestir', 'ajuda_adl', 'dificuldade_compras', 'dificuldade_medico', 'ajuda_iadl', 'queda_12m', 'usa_internet', 'usa_celular', 'depressao_diag', 'vacina_influenza', 'autoavaliacao_saude', 'peso_real', 'altura', 'atividade_fisica', 'fumante_atual', 'hipertensao', 'diabetes', 'doenca_cardiaca', 'avc', 'doenca_respiratoria', 'cancer', 'num_medicamentos', 'id_individuo', 'area_urbana', 'peso_amostral', 'regiao', 'imc', 'multimorbidity_count', 'multimorb_cat', 'adl_score', 'iadl_score', 'functional_raw', 'functional_score', 'dependencia_SUS', 'cobertura_influenza', 'autoav_z', 'multimorb_z', 'functional_z', 'health_score_raw', 'health_score']


In [3]:
# quero ver todas as colunas sem supressao visual
pd.set_option('display.max_columns', None)
df.head(5)

,uf,estrato,upa,id_domicilio,num_pessoas_domicilio,area_metropolitana,mora_sozinho,sexo,idade,raca_cor,situacao_ocupacional,anos_estudo,renda_percapita,possui_plano_saude,consulta_12m,internacoes_12m,atendimento_sus,dificuldade_alimentar,dificuldade_banho,dificuldade_vestir,ajuda_adl,dificuldade_compras,dificuldade_medico,ajuda_iadl,queda_12m,usa_internet,usa_celular,depressao_diag,vacina_influenza,autoavaliacao_saude,peso_real,altura,atividade_fisica,fumante_atual,hipertensao,diabetes,doenca_cardiaca,avc,doenca_respiratoria,cancer,num_medicamentos,id_individuo,area_urbana,peso_amostral,regiao,imc,multimorbidity_count,multimorb_cat,adl_score,iadl_score,functional_raw,functional_score,dependencia_SUS,cobertura_influenza,autoav_z,multimorb_z,functional_z,health_score_raw,health_score
0,Rondônia,1110011,110000016,1,1,1,Cônjuge ou companheiro(a),Masculino,69.0,Parda,Ocupado,6.0,1200.0,Não,Não se aplica,1,Não,Não consegue de modo algum,Não consegue de modo algum,Muita dificuldade,Não,Não consegue de modo algum,Muita dificuldade,Não,Não,Não,Não,Não sabe,Não sabe,NaN,NaN,NaN,Não sabe,Não,Não sabe,Não sabe,Não sabe,Não sabe,Não sabe,Não sabe,2.0,85.589085,NaN,NaN,NaN,149.253731,0,0,11,7,18,0.10,0,0,NaN,-1.027069,0.187726,NaN,NaN
1,Rondônia,1110011,110000016,1,1,1,Outro morador,Feminino,78.0,Parda,Não aplicável,6.0,1200.0,Não,Não se aplica,1,Não,Não consegue de modo algum,Não consegue de modo algum,Não consegue de modo algum,Não,Não consegue de modo algum,Nenhuma dificuldade,Sim,Sim,Não,Não,Não sabe,Não sabe,NaN,NaN,NaN,Não sabe,Não,Não sabe,Não sabe,Não sabe,Não sabe,Não sabe,Não sabe,2.0,85.589085,NaN,NaN,NaN,149.253731,0,0,12,5,17,0.15,0,0,NaN,-1.027069,0.522010,NaN,NaN
2,Rondônia,1110011,110000016,1,1,1,Pessoa de referência,Masculino,81.0,Parda,Não aplicável,5.0,1200.0,Não,Não sabe,1,Não,Nenhuma dificuldade,Muita dificuldade,Muita dificuldade,Sim,Nenhuma dificuldade,Nenhuma dificuldade,Sim,Sim,Sim,Não,Não sabe,Não,Muito boa,81.0,0.81,Não,"Não, mas já fumou",Sim,Não,Não,Não sabe,Não,Não sabe,2.0,85.589085,0,117.758255,Norte,123.456790,11,3+,7,2,9,0.55,0,2,-0.344963,1.200084,3.196276,3.167954,0.744213
3,Rondônia,1110011,110000016,1,1,1,Pessoa de referência,Masculino,81.0,Parda,Desocupado,5.0,1200.0,Não,Não se aplica,1,Não,Não consegue de modo algum,Não consegue de modo algum,Não consegue de modo algum,NaN,Não consegue de modo algum,Muita dificuldade,Não,Não,Sim,Não,Não sabe,Não,Muito boa,75.0,0.75,Não,"Não, mas já fumou",Sim,Sim,Não,Não sabe,Não,Não sabe,2.0,85.589085,0,117.758255,Norte,133.333333,9,3+,12,7,19,0.05,0,2,-0.344963,0.795147,-0.146557,-0.559990,0.305083
4,Rondônia,1110011,110000016,1,1,1,Pessoa de referência,Feminino,72.0,Parda,Fora da força de trabalho,6.0,1200.0,Não,Não se aplica,1,Não,Muita dificuldade,Muita dificuldade,Muita dificuldade,Não,Não consegue de modo algum,Muita dificuldade,Sim,Não,Não,Não,Não sabe,Não sabe,NaN,NaN,NaN,Não sabe,Não,Não sabe,Não sabe,Não sabe,Não sabe,Não sabe,Não sabe,2.0,85.589085,NaN,NaN,NaN,149.253731,0,0,9,7,16,0.20,0,0,NaN,-1.027069,0.856293,NaN,NaN


In [4]:
# ===============================================================================
# ANÁLISE AVANÇADA DO ÍNDICE DE ENVELHECIMENTO ATIVO (AAI)
# PNS 2019 - Versão Corrigida e Metodologicamente Robusta
# ===============================================================================
# Autor: Análise Senior - Metodologia Survey-Aware
# Data: Outubro 2024
# Objetivo: AAI municipal com inferência válida e intervenções acionáveis
# ===============================================================================
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# Pacotes opcionais
try:
    import geopandas as gpd
    from esda.moran import Moran, Moran_Local
    import libpysal
    SPATIAL_AVAILABLE = True
except:
    SPATIAL_AVAILABLE = False
    print(" Pacotes espaciais não disponíveis (instale: geopandas, esda, libpysal)")

try:
    import shap
    SHAP_AVAILABLE = True
except:
    SHAP_AVAILABLE = False
    print(" SHAP não disponível (instale: shap)")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
plt.style.use('seaborn-v0_8-whitegrid')

print("="*80)
print("BIBLIOTECAS CARREGADAS COM SUCESSO")
print("="*80)

BIBLIOTECAS CARREGADAS COM SUCESSO


In [5]:
# ===============================================================================
# SEÇÃO 1: CONFIGURAÇÃO E CARREGAMENTO
# ===============================================================================

# 🔧 AJUSTE OS CAMINHOS AQUI
DATA_PATH = "data/processed/pns_2019_pandas.csv"
SHAPEFILE_PATH = "data/processed/BR_municipios_2019.shp"
DATASUS_PATH = "data/processed/datasus_facilities.csv"
OUTPUT_DIR = Path("./outputs_aai")
OUTPUT_DIR.mkdir(exist_ok=True)

# Configurações metodológicas
MIN_N_MUNICIPAL = 30  # Mínimo de observações para estimativas municipais confiáveis
N_BOOTSTRAP = 500     # Iterações para CIs
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print(f"\nCarregando dados de: {DATA_PATH}")
try:
    df = pd.read_csv(DATA_PATH, low_memory=False)
    print(f"Dataset carregado: {df.shape[0]:,} linhas × {df.shape[1]} colunas")
except FileNotFoundError:
    print("Erro: Arquivo não encontrado. Ajuste DATA_PATH")
    raise


Carregando dados de: data/processed/pns_2019_pandas.csv
Dataset carregado: 43,554 linhas × 59 colunas
Dataset carregado: 43,554 linhas × 59 colunas


In [6]:
# ===============================================================================
# SEÇÃO 2: PADRONIZAÇÃO DE VARIÁVEIS CRÍTICAS
# ===============================================================================

print("\n" + "="*80)
print(" PADRONIZAÇÃO DE VARIÁVEIS")
print("="*80)

for col_name in ['peso', 'peso_amostral', 'peso_exp', 'weight']:
    if col_name in df.columns:
        WEIGHT_COL = col_name
        print(f"Coluna de peso identificada: '{WEIGHT_COL}'")
        break

if WEIGHT_COL is None:
    raise SystemExit("Erro critico: Coluna de peso não encontrada. Estimativas não-ponderadas são invalidas.")

# Validar variáveis essenciais
essential_vars = {
    'idade': 'Idade do indivíduo',
    'codmun': 'Código do município',
    'uf': 'Unidade Federativa',
    'estrato': 'Estrato amostral'
}

print("\nValidação de variáveis essenciais:")
missing_essentials = []
for var, desc in essential_vars.items():
    if var in df.columns:
        print(f"  ✓ {var:12s} - {desc}")
    else:
        print(f"  ✗ {var:12s} - {desc} [FALTANDO]")
        missing_essentials.append(var)

if missing_essentials:
    print(f"\nAtenção: Variáveis ausentes podem limitar análises: {missing_essentials}")


 PADRONIZAÇÃO DE VARIÁVEIS
Coluna de peso identificada: 'peso_amostral'

Validação de variáveis essenciais:
  ✓ idade        - Idade do indivíduo
  ✗ codmun       - Código do município [FALTANDO]
  ✓ uf           - Unidade Federativa
  ✓ estrato      - Estrato amostral

Atenção: Variáveis ausentes podem limitar análises: ['codmun']


In [7]:
# Derivar codmun se não existir
if 'codmun' not in df.columns:
    if 'upa' in df.columns:
        df['codmun'] = df['upa'].astype(str).str[:6].astype(int)
        print(" Coluna 'codmun' derivada de 'upa'")
    else:
        raise SystemExit(" ERRO: Não é possível derivar 'codmun'. Coluna 'upa' não encontrada.")

 Coluna 'codmun' derivada de 'upa'


In [8]:
# ===============================================================================
# SEÇÃO 3: FILTROS E PREPARAÇÃO
# ===============================================================================

print("\n" + "="*80)
print(" APLICANDO FILTROS")
print("="*80)

n_original = len(df)
print(f"Registros originais: {n_original:,}")

# Filtro 60+
if 'idade' in df.columns:
    df = df[df['idade'] >= 60].copy()
    print(f"Apos filtro 60+: {len(df):,} registros ({len(df)/n_original*100:.1f}%)")
else:
    raise SystemExit("Erro: Coluna 'idade' não encontrada")

# Verificar pesos válidos
invalid_weights = df[WEIGHT_COL].isnull().sum()
if invalid_weights > 0:
    print(f"Removendo {invalid_weights} registros com peso missing")
    df = df[df[WEIGHT_COL].notna()].copy()

# Criar faixa etária
df['faixa_etaria'] = pd.cut(df['idade'], 
                             bins=[60, 70, 80, 120], 
                             labels=['60-69', '70-79', '80+'],
                             include_lowest=True)

print(f"\nDistribuição etária (ponderada):")
for age_group in ['60-69', '70-79', '80+']:
    pop = df[df['faixa_etaria'] == age_group][WEIGHT_COL].sum()
    pct = pop / df[WEIGHT_COL].sum() * 100
    print(f"  {age_group}: {pop:,.0f} ({pct:.1f}%)")


 APLICANDO FILTROS
Registros originais: 43,554
Apos filtro 60+: 43,554 registros (100.0%)
Removendo 20826 registros com peso missing

Distribuição etária (ponderada):
  60-69: 20,778,673 (60.4%)
  70-79: 9,720,855 (28.3%)
  80+: 3,899,325 (11.3%)


In [9]:
# ===============================================================================
# SEÇÃO 4: FUNÇÕES PONDERADAS COM BOOTSTRAP
# ===============================================================================

print("\n" + "="*80)
print(" DEFININDO FUNÇÕES ESTATÍSTICAS SURVEY-AWARE")
print("="*80)

def weighted_mean(data, col, weight_col=WEIGHT_COL):
    """Média ponderada com tratamento de missing"""
    valid = data[[col, weight_col]].dropna()
    if len(valid) == 0:
        return np.nan
    return (valid[col] * valid[weight_col]).sum() / valid[weight_col].sum()

def weighted_std(data, col, weight_col=WEIGHT_COL):
    """Desvio padrão ponderado"""
    valid = data[[col, weight_col]].dropna()
    if len(valid) == 0:
        return np.nan
    mean = weighted_mean(data, col, weight_col)
    variance = ((valid[col] - mean)**2 * valid[weight_col]).sum() / valid[weight_col].sum()
    return np.sqrt(variance)

def weighted_quantile(data, col, q, weight_col=WEIGHT_COL):
    """Quantil ponderado"""
    valid = data[[col, weight_col]].dropna().sort_values(col)
    if len(valid) == 0:
        return np.nan
    cumsum = valid[weight_col].cumsum()
    cutoff = valid[weight_col].sum() * q
    return valid[col].iloc[(cumsum >= cutoff).argmax()]

def weighted_mean_bootstrap_ci(data, col, weight_col=WEIGHT_COL, n_boot=N_BOOTSTRAP, ci=95):
    """
    CORREÇÃO CRÍTICA: Bootstrap para intervalos de confiança
    Respeita estrutura de pesos amostrais
    """
    valid = data[[col, weight_col]].dropna().reset_index(drop=True)
    if len(valid) < 10:
        return np.nan, np.nan, np.nan
    
    estimates = []
    for _ in range(n_boot):
        idx = np.random.choice(len(valid), size=len(valid), replace=True)
        boot_data = valid.iloc[idx]
        boot_mean = (boot_data[col] * boot_data[weight_col]).sum() / boot_data[weight_col].sum()
        estimates.append(boot_mean)
    
    point_est = weighted_mean(data, col, weight_col)
    lower = np.percentile(estimates, (100-ci)/2)
    upper = np.percentile(estimates, 100-(100-ci)/2)
    
    return point_est, lower, upper

print("Funções definidas com bootstrap para CIs")


 DEFININDO FUNÇÕES ESTATÍSTICAS SURVEY-AWARE
Funções definidas com bootstrap para CIs


In [10]:
# ===============================================================================
# SEÇÃO 5: ANÁLISE DE MISSINGNESS
# ===============================================================================

print("\n" + "="*80)
print(" ANÁLISE DE DADOS FALTANTES")
print("="*80)

# Identificar domínios disponíveis
ALL_DOMAINS = ['health_score', 'functional_score', 'participation_score', 
               'econ_score', 'access_score']

print("\nDisponibilidade de domínios:")
available_domains = []
for domain in ALL_DOMAINS:
    if domain in df.columns:
        missing_pct = df[domain].isnull().mean() * 100
        print(f"  ✓ {domain:25s} - {missing_pct:.1f}% missing")
        available_domains.append(domain)
    else:
        print(f"  ✗ {domain:25s} - NÃO DISPONÍVEL")

if len(available_domains) == 0:
    raise SystemExit("Erro critico: Nenhum domínio encontrado. Verifique nomes das colunas.")

print(f"\nTotal de domínios disponíveis: {len(available_domains)}/{len(ALL_DOMAINS)}")

# Missing por faixa etária (detectar viés)
key_vars = ['renda', 'anos_estudo', 'uso_internet', 'plano']
print("\nPadrão de missing por idade (potencial viés):")

missing_by_age = []
for var in key_vars:
    if var in df.columns:
        for age_group in ['60-69', '70-79', '80+']:
            subset = df[df['faixa_etaria'] == age_group]
            miss_pct = subset[var].isnull().mean() * 100
            missing_by_age.append({
                'Variável': var,
                'Faixa': age_group,
                'Missing %': f"{miss_pct:.1f}%"
            })

if missing_by_age:
    miss_df = pd.DataFrame(missing_by_age).pivot(index='Variável', 
                                                  columns='Faixa', 
                                                  values='Missing %')
    print(miss_df)

# ===============================================================================
# CONSTRUÇÃO DOS SCORES FALTANTES
# ===============================================================================

print("\n" + "="*80)
print("CONSTRUÇÃO DOS SCORES FALTANTES")
print("="*80)

# Função para normalizar variáveis (0-100)
def normalize_score(series, reverse=False):
    """Normaliza série para escala 0-100"""
    if reverse:
        series = -series
    min_val = series.min()
    max_val = series.max()
    if max_val == min_val:
        return pd.Series([50] * len(series), index=series.index)
    normalized = (series - min_val) / (max_val - min_val) * 100
    return normalized

# 1. PARTICIPATION_SCORE (acesso à internet + celular)
print("\nConstruindo participation_score...")
if 'participation_score' not in df.columns:
    participation_vars = []
    
    # Internet
    if 'usa_internet' in df.columns:
        internet_score = df['usa_internet'].map({'Sim': 1, 'Não': 0}).fillna(0) * 100
        participation_vars.append(internet_score)
        print("   ✓ Internet access incluído")
    
    # Celular
    if 'usa_celular' in df.columns:
        celular_score = df['usa_celular'].map({'Sim': 1, 'Não': 0}).fillna(0) * 100
        participation_vars.append(celular_score)
        print("   ✓ Celular access incluído")
    
    if participation_vars:
        df['participation_score'] = pd.concat(participation_vars, axis=1).mean(axis=1)
        print(f"   → participation_score criado (média de {len(participation_vars)} indicadores)")
    else:
        print("    Nenhuma variável de participação disponível")
else:
    print("   ✓ participation_score já existe")

# 2. ECON_SCORE (educação + renda)
print("\nConstruindo econ_score...")
if 'econ_score' not in df.columns:
    econ_vars = []
    
    # Educação
    if 'anos_estudo' in df.columns:
        educ_score = normalize_score(df['anos_estudo'].fillna(df['anos_estudo'].median()))
        econ_vars.append(educ_score)
        print("   ✓ Educação incluída")
    
    # Renda
    if 'renda' in df.columns:
        renda_score = normalize_score(df['renda'].fillna(df['renda'].median()))
        econ_vars.append(renda_score)
        print("   ✓ Renda incluída")
    
    if econ_vars:
        df['econ_score'] = pd.concat(econ_vars, axis=1).mean(axis=1)
        print(f"   → econ_score criado (média de {len(econ_vars)} indicadores)")
    else:
        print("    Nenhuma variável econômica disponível")
else:
    print("   ✓ econ_score já existe")

# 3. ACCESS_SCORE (acesso a serviços de saúde)
print("\nConstruindo access_score...")
if 'access_score' not in df.columns:
    access_vars = []
    
    # Plano de saúde
    if 'possui_plano_saude' in df.columns:
        plano_score = df['possui_plano_saude'].map({'Sim': 1, 'Não': 0}).fillna(0) * 100
        access_vars.append(plano_score)
        print("   ✓ Plano de saúde incluído")
    
    # Consulta médica recente
    if 'consulta_12m' in df.columns:
        consulta_score = df['consulta_12m'].map({'Sim': 1, 'Não': 0}).fillna(0) * 100
        access_vars.append(consulta_score)
        print("   ✓ Consulta médica incluída")
    
    if access_vars:
        df['access_score'] = pd.concat(access_vars, axis=1).mean(axis=1)
        print(f"   → access_score criado (média de {len(access_vars)} indicadores)")
    else:
        print("    Nenhuma variável de acesso disponível")
else:
    print("   ✓ access_score já existe")

# Validação dos scores criados
print("\nValidação dos scores criados:")
for score in ['participation_score', 'econ_score', 'access_score']:
    if score in df.columns:
        mean_val = df[score].mean()
        missing_pct = df[score].isnull().mean() * 100
        print(f"   ✓ {score:20s}: média={mean_val:.1f}, missing={missing_pct:.1f}%")
    else:
        print(f"   ✗ {score:20s}: não criado")

print("\nConstrução de scores concluída!")


 ANÁLISE DE DADOS FALTANTES

Disponibilidade de domínios:
  ✓ health_score              - 0.0% missing
  ✓ functional_score          - 0.0% missing
  ✗ participation_score       - NÃO DISPONÍVEL
  ✗ econ_score                - NÃO DISPONÍVEL
  ✗ access_score              - NÃO DISPONÍVEL

Total de domínios disponíveis: 2/5

Padrão de missing por idade (potencial viés):
Faixa       60-69 70-79   80+
Variável                     
anos_estudo  0.0%  0.0%  0.0%

CONSTRUÇÃO DOS SCORES FALTANTES

Construindo participation_score...
   ✓ Internet access incluído
   ✓ Celular access incluído
   → participation_score criado (média de 2 indicadores)

Construindo econ_score...
   ✓ Educação incluída
   → econ_score criado (média de 1 indicadores)

Construindo access_score...
   ✓ Plano de saúde incluído
   ✓ Consulta médica incluída
   → access_score criado (média de 2 indicadores)

Validação dos scores criados:
   ✓ participation_score : média=51.4, missing=0.0%
   ✓ econ_score          : média=

In [11]:
# ===============================================================================
# SEÇÃO 6: CONSTRUÇÃO DO AAI_TOTAL (CORRIGIDA)
# ===============================================================================

print("\n" + "="*80)
print(" CONSTRUÇÃO DO AAI_TOTAL")
print("="*80)

# CORREÇÃO CRÍTICA 2: AAI usando domínios disponíveis
df['AAI_total'] = df[available_domains].mean(axis=1)
print(f"AAI_total criado usando {len(available_domains)} domínios:")
for dom in available_domains:
    print(f"   • {dom}")

# Estatísticas descritivas do AAI
print("\nEstatísticas do AAI_TOTAL (ponderadas):")
aai_mean, aai_lower, aai_upper = weighted_mean_bootstrap_ci(df, 'AAI_total')
print(f"  Média: {aai_mean:.2f} [95% CI: {aai_lower:.2f} - {aai_upper:.2f}]")
print(f"  DP:    {weighted_std(df, 'AAI_total'):.2f}")
print(f"  P25:   {weighted_quantile(df, 'AAI_total', 0.25):.2f}")
print(f"  P50:   {weighted_quantile(df, 'AAI_total', 0.50):.2f}")
print(f"  P75:   {weighted_quantile(df, 'AAI_total', 0.75):.2f}")


 CONSTRUÇÃO DO AAI_TOTAL
AAI_total criado usando 2 domínios:
   • health_score
   • functional_score

Estatísticas do AAI_TOTAL (ponderadas):
  Média: 0.18 [95% CI: 0.17 - 0.18]
  DP:    0.13
  P25:   0.12
  P50:   0.13
  P75:   0.17
  Média: 0.18 [95% CI: 0.17 - 0.18]
  DP:    0.13
  P25:   0.12
  P50:   0.13
  P75:   0.17


In [12]:
# ===============================================================================
# SEÇÃO 7: AGREGAÇÃO MUNICIPAL (COM CONTROLE DE QUALIDADE)
# ===============================================================================

print("\n" + "="*80)
print(" AGREGAÇÃO MUNICIPAL")
print("="*80)

def aggregate_municipal_robust(group):
    """Agregação municipal com CIs e flags de qualidade"""
    results = {
        'n_obs': len(group),
        'pop_weight_sum': group[WEIGHT_COL].sum()
    }
    
    # Calcular AAI com CI
    if 'AAI_total' in group.columns:
        aai_mean, aai_lower, aai_upper = weighted_mean_bootstrap_ci(group, 'AAI_total')
        results['AAI_total'] = aai_mean
        results['AAI_ci_lower'] = aai_lower
        results['AAI_ci_upper'] = aai_upper
        results['AAI_ci_width'] = aai_upper - aai_lower
    
    # Domínios individuais
    for domain in available_domains:
        if domain in group.columns:
            results[domain] = weighted_mean(group, domain)
    
    # Flags de qualidade
    results['reliable'] = len(group) >= MIN_N_MUNICIPAL
    
    return pd.Series(results)

print(f"Agregando por município (mínimo n={MIN_N_MUNICIPAL})...")
municipal_scores = df.groupby('codmun').apply(aggregate_municipal_robust).reset_index()

# Adicionar UF
if 'uf' in df.columns:
    mun_uf = df.groupby('codmun')['uf'].first().reset_index()
    municipal_scores = municipal_scores.merge(mun_uf, on='codmun', how='left')

# Estatísticas de qualidade
n_total_mun = len(municipal_scores)
n_reliable = municipal_scores['reliable'].sum()
print(f"\n{n_total_mun} municípios processados")
print(f"   • {n_reliable} municípios com n≥{MIN_N_MUNICIPAL} (confiáveis)")
print(f"   • {n_total_mun - n_reliable} municípios com n<{MIN_N_MUNICIPAL} (requerem SAE)")

# Salvar
output_path = OUTPUT_DIR / "municipal_scores_with_ci.csv"
municipal_scores.to_csv(output_path, index=False)
print(f"\nSalvo: {output_path}")


 AGREGAÇÃO MUNICIPAL
Agregando por município (mínimo n=30)...

2794 municípios processados
   • 46 municípios com n≥30 (confiáveis)
   • 2748 municípios com n<30 (requerem SAE)

Salvo: outputs_aai\municipal_scores_with_ci.csv

2794 municípios processados
   • 46 municípios com n≥30 (confiáveis)
   • 2748 municípios com n<30 (requerem SAE)

Salvo: outputs_aai\municipal_scores_with_ci.csv


## Resultados Principais

Nas seções seguintes, apresentamos os principais achados da análise:

1. **Preparação dos Dados**: Limpeza e validação dos dados da PNS 2019
2. **Construção do AAI**: Índice nacional e estatísticas descritivas
3. **Agregação Municipal**: Scores por município com controle de qualidade
4. **Identificação de Hotspots**: Municípios prioritários para intervenção
5. **Análise de Desigualdades**: Comparação entre subgrupos demográficos
6. **Perfis de Envelhecimento**: Clustering para identificação de padrões
7. **Modelagem Preditiva**: Drivers de vulnerabilidade identificados
8. **Análise de Mediação**: Efeitos indiretos sobre o envelhecimento ativo
9. **Análise Espacial**: Padrões geográficos e autocorrelação
10. **Policy Brief**: Recomendações para gestores públicos

Todos os resultados consideram o desenho amostral complexo da PNS, utilizando pesos amostrais e intervalos de confiança.

## 1. Preparando o Terreno: Organizando Nossos Dados

Antes de qualquer análise, precisamos garantir que nossos dados estão limpos e organizados. Nesta etapa, estamos fazendo uma 'faxina':

* **Filtramos os dados** para focar apenas nas pessoas com 60 anos ou mais, que são o nosso público de interesse.
* **Verificamos informações essenciais**, como o município de residência e a idade de cada pessoa.
* **Validamos os 'pesos amostrais'**: um fator de correção crucial que nos permite dizer que os resultados desta pesquisa representam a população idosa de todo o Brasil, e não apenas as pessoas que foram entrevistadas.

Pense nos pesos amostrais como "multiplicadores": algumas pessoas representam milhares de outras com características semelhantes. Sem eles, nossos resultados seriam enviesados.

## 2. Construindo o Termômetro: O Índice de Envelhecimento Ativo (AAI)

Para medir o 'envelhecimento ativo', não podemos olhar para um único fator. Por isso, criamos um índice, o AAI, que funciona como uma nota final composta por várias 'matérias': saúde, participação social, segurança econômica, etc.

Nesta seção, nós:
1. **Calculamos as notas** para cada uma dessas 'matérias' (os domínios).
2. **Combinamos tudo** para criar a nota final: o **AAI_total**.
3. **Usamos uma técnica estatística (bootstrap)** para garantir que nossas médias não sejam apenas um número, mas uma 'faixa de valores prováveis' (o intervalo de confiança), o que nos dá muito mais segurança nos resultados.

O resultado é um panorama nacional do envelhecimento ativo, com indicadores robustos que podem ser comparados entre regiões e grupos populacionais.

## 3. Do Indivíduo para a Cidade: Calculando a Nota de Cada Município

Agora que cada idoso tem sua 'nota' de envelhecimento ativo, o próximo passo é calcular a média para cada município do Brasil. Isso nos permite comparar as cidades e ver onde a qualidade de vida na terceira idade é maior ou menor.

**Ponto de atenção:** Para que a nota de um município seja confiável, exigimos um número mínimo de entrevistados naquela cidade. Cidades com poucos participantes são sinalizadas, pois suas médias podem não ser tão precisas.

O resultado é um mapa municipal do envelhecimento ativo, fundamental para orientar onde investir recursos públicos.

## 4. Onde Devemos Focar? Identificando os Municípios Prioritários

Com a nota de cada cidade em mãos, podemos criar um ranking. Aqui, nosso objetivo é responder a uma pergunta fundamental para a gestão pública: **"Quais municípios mais precisam de ajuda?"**

Para isso, filtramos apenas os municípios com dados confiáveis e, em seguida, identificamos o grupo dos **20% com as piores notas** no Índice de Envelhecimento Ativo (AAI).

Esta lista de "municípios prioritários" é um dos resultados mais importantes do nosso estudo, pois serve como um guia para direcionar recursos, programas sociais e políticas de saúde para as áreas mais críticas do país.

## 5. Há Diferenças entre Grupos? Analisando Desigualdades

O envelhecimento ativo pode variar muito dependendo de características pessoais como sexo, raça, escolaridade ou local de residência (urbano/rural).

Nesta seção, comparamos as notas do AAI entre diferentes grupos demográficos para identificar desigualdades. Por exemplo:
- As mulheres idosas têm notas diferentes dos homens?
- Há diferenças por nível de escolaridade?
- Idosos em áreas rurais enfrentam mais desafios?

Essas análises são cruciais para criar políticas mais justas e direcionadas, garantindo que nenhum grupo seja deixado para trás.

## 6. Existem 'Grupos' de Idosos? Criando Perfis de Envelhecimento

Será que todos os idosos envelhecem da mesma forma? Provavelmente não. Nesta seção, usamos uma técnica que funciona como um 'chapéu seletor': ela agrupa os idosos em perfis distintos com base em suas características de saúde, renda, educação e participação.

O resultado é a identificação de, por exemplo, um grupo de 'idosos ativos e conectados' e outro de 'idosos frágeis e com dificuldades de acesso'. Entender esses perfis ajuda a criar ações mais personalizadas e eficazes.

Usamos uma técnica chamada 'clustering' que encontra padrões naturais nos dados, sem precisar dizer previamente quantos grupos queremos.

## 7. Quais Fatores Mais Influenciam? Descobrindo os Drivers de Vulnerabilidade

Por que alguns idosos têm uma nota de envelhecimento ativo tão baixa? Para responder a isso, construímos um modelo preditivo.

Pense nele como um detetive que analisa milhares de casos para descobrir quais 'pistas' (características como escolaridade, renda, acesso à internet) estão mais fortemente ligadas à vulnerabilidade. O resultado nos mostra os fatores mais importantes que, se melhorados, podem ter o maior impacto positivo na vida da população idosa.

Usamos técnicas avançadas como Random Forest e SHAP para garantir que as descobertas sejam robustas e interpretáveis.

## 8. Há Efeitos Indiretos? Análise de Mediação

Às vezes, os fatores não influenciam diretamente o envelhecimento ativo, mas sim indiretamente, através de outros fatores intermediários.

Por exemplo: a renda pode afetar a participação social, que por sua vez afeta o envelhecimento ativo. Nesta seção, investigamos esses 'efeitos em cascata' usando uma técnica chamada análise de mediação.

Isso nos ajuda a entender melhor como as intervenções funcionam: melhorar a renda pode ter benefícios adicionais através da participação social.

## 9. Há Padrões Geográficos? Análise Espacial

Será que municípios vizinhos têm notas de envelhecimento ativo semelhantes? Ou há padrões regionais que influenciam o envelhecimento?

Nesta seção, usamos técnicas de análise espacial para investigar:
- **Autocorrelação**: Se municípios próximos tendem a ter notas parecidas
- **Clusters espaciais**: Grupos de municípios com características similares

Isso é importante porque fatores como clima, cultura regional ou políticas locais podem criar padrões geográficos que precisam ser considerados no planejamento.

## 10. Comunicando os Resultados: Policy Brief Automatizado

Agora que temos todos os insights, precisamos comunicá-los de forma clara e acionável. Nesta seção, geramos automaticamente um documento executivo (policy brief) que resume:

- Os principais indicadores nacionais
- Os municípios prioritários
- As desigualdades identificadas
- Os fatores de risco mais importantes
- Recomendações práticas para gestores públicos

Este documento é projetado para ser lido rapidamente por tomadores de decisão, com linguagem clara e foco em ações concretas.

## 11. Visualizando os Dados: Gráficos e Mapas

Uma imagem vale mais que mil palavras. Nesta seção, criamos visualizações que facilitam o entendimento dos dados:

- **Distribuições dos domínios**: Como cada componente do AAI se comporta na população
- **AAI por faixa etária**: Como o envelhecimento ativo varia com a idade
- **Importância dos fatores**: Quais características mais influenciam a vulnerabilidade

Esses gráficos são salvos automaticamente e podem ser usados em apresentações, relatórios ou publicações.

## 12. Verificando a Qualidade: Checklist de Validação

Antes de finalizar, precisamos garantir que nossa análise está correta e completa. Nesta seção, executamos um checklist automático que verifica:

- Se todos os filtros foram aplicados corretamente
- Se os pesos amostrais estão sendo usados
- Se os cálculos estão consistentes
- Se todos os arquivos de saída foram gerados

Isso garante que os resultados são confiáveis e reproduzíveis.

## 13. Conclusões e Recomendações: O Que Tudo Isso Significa?

Chegamos ao final da nossa jornada pelos dados. Nesta seção, sintetizamos os principais insights e traduzimos em recomendações práticas:

- **O que aprendemos** sobre o envelhecimento ativo no Brasil
- **Quais são as prioridades** para ação governamental
- **Como os resultados podem orientar** políticas públicas
- **Quais limitações** devemos considerar

Este sumário final é escrito em linguagem acessível, focando no impacto social e nas oportunidades de melhoria.

## 14. Preparando para o Futuro: Exportando Datasets

Para que outros pesquisadores ou gestores possam continuar o trabalho, nesta seção exportamos todos os datasets processados:

- **Dados individuais** com scores calculados e clusters identificados
- **Sumário executivo** em Excel com múltiplas abas
- **Arquivos geoespaciais** para mapeamento

Isso garante que o conhecimento gerado possa ser reutilizado e expandido por outros projetos.

### O que conseguimos?

✅ Dados limpos e validados da PNS 2019  
✅ Índice de Envelhecimento Ativo (AAI) construído com rigor estatístico  
✅ Panorama municipal com identificação de hotspots  
✅ Perfis de idosos identificados  
✅ Fatores de vulnerabilidade descobertos  
✅ Análises espaciais e de mediação  
✅ Policy brief automatizado  
✅ Visualizações e datasets exportados  

### Próximos passos sugeridos:

1. **Revisar os resultados** no policy brief gerado
2. **Explorar os mapas** para entender padrões regionais  
3. **Usar os datasets exportados** para análises adicionais
4. **Compartilhar os insights** com stakeholders relevantes

Esta análise serve como base sólida para políticas públicas mais informadas e eficazes para a população idosa brasileira.

---

*Obrigado por acompanhar esta jornada pelos dados!*

In [13]:
# ===============================================================================
# SEÇÃO 4: IDENTIFICAÇÃO DE HOTSPOTS (CORRIGIDA)
# ===============================================================================

print("\n" + "="*80)
print("IDENTIFICAÇÃO DE MUNICÍPIOS VULNERÁVEIS")
print("="*80)

# CORREÇÃO CRÍTICA 3: Filtrar apenas municípios confiáveis
mun_reliable = municipal_scores[municipal_scores['reliable'] == True].copy()
print(f"Analisando {len(mun_reliable)} municípios com dados confiáveis...")

# Calcular percentil 20 (bottom 20%)
threshold_p20 = mun_reliable['AAI_total'].quantile(0.20)
worst_20pct = mun_reliable[mun_reliable['AAI_total'] <= threshold_p20].sort_values('AAI_total')

print(f"\nMunicípios prioritários (Bottom 20%)")
print(f"   Threshold AAI: {threshold_p20:.2f}")
print(f"   Total municípios: {len(worst_20pct)}")
print(f"   População afetada: {worst_20pct['pop_weight_sum'].sum():,.0f}")

# Top 20 piores
print("\n TOP 20 MUNICÍPIOS MAIS VULNERÁVEIS:")
display_cols = ['codmun', 'uf', 'AAI_total', 'AAI_ci_lower', 'AAI_ci_upper', 
                'health_score', 'n_obs']
display_cols = [c for c in display_cols if c in worst_20pct.columns]
print(worst_20pct[display_cols].head(20).to_string(index=False))

# Salvar lista prioritária
priority_path = OUTPUT_DIR / "priority_municipalities_bottom20.csv"
worst_20pct.to_csv(priority_path, index=False)
print(f"\n Lista prioritária salva: {priority_path}")


IDENTIFICAÇÃO DE MUNICÍPIOS VULNERÁVEIS
Analisando 46 municípios com dados confiáveis...

Municípios prioritários (Bottom 20%)
   Threshold AAI: 0.15
   Total municípios: 10
   População afetada: 135,259

 TOP 20 MUNICÍPIOS MAIS VULNERÁVEIS:
 codmun             uf  AAI_total  AAI_ci_lower  AAI_ci_upper  health_score  n_obs
 160000          Amapá   0.126557      0.102618      0.154344      0.223901     31
 280021        Sergipe   0.130634      0.116387      0.148204      0.240169     32
 120001           Acre   0.135348      0.117091      0.155835      0.246038     44
 140003        Roraima   0.136175      0.121368      0.152217      0.240602     38
 170026      Tocantins   0.136778      0.099226      0.184577      0.234800     34
 140006        Roraima   0.139188      0.125600      0.153537      0.259095     40
 320011 Espírito Santo   0.142082      0.126613      0.163515      0.266439     30
 170005      Tocantins   0.144686      0.115648      0.182890      0.253534     30
 420024 Sa

In [14]:
# ===============================================================================
# SEÇÃO 5: DESIGUALDADES POR SUBGRUPO
# ===============================================================================

print("\n" + "="*80)
print("DESIGUALDADES POR SUBGRUPO")
print("="*80)

subgroup_vars = ['sexo', 'raca_cor', 'escolaridade', 'urbano_rural']

for var in subgroup_vars:
    if var in df.columns:
        print(f"\n {var.upper().replace('_', ' ')}:")
        
        subgroup_stats = []
        for category in df[var].dropna().unique():
            subset = df[df[var] == category]
            if len(subset) > 0:
                aai_mean = weighted_mean(subset, 'AAI_total')
                health_mean = weighted_mean(subset, 'health_score') if 'health_score' in df.columns else np.nan
                n_weighted = subset[WEIGHT_COL].sum()
                
                subgroup_stats.append({
                    'Categoria': category,
                    'AAI': f"{aai_mean:.2f}",
                    'Health': f"{health_mean:.2f}" if not np.isnan(health_mean) else "N/A",
                    'N (ponderado)': f"{n_weighted:,.0f}"
                })
        
        if subgroup_stats:
            print(pd.DataFrame(subgroup_stats).to_string(index=False))


DESIGUALDADES POR SUBGRUPO

 SEXO:
Categoria  AAI Health N (ponderado)
Masculino 0.16   0.28    14,901,688
 Feminino 0.19   0.30    19,497,165

 RACA COR:
Categoria  AAI Health N (ponderado)
    Parda 0.17   0.28    12,861,842
    Preta 0.18   0.29     3,535,497
   Branca 0.18   0.29    17,375,149
  Amarela 0.18   0.30       440,363
 Indígena 0.18   0.29       185,496
 Ignorado 0.12   0.24           507


In [15]:
# ===============================================================================
# SEÇÃO 6: CLUSTERING COM IMPUTAÇÃO (CORRIGIDO)
# ===============================================================================

print("\n" + "="*80)
print("PERFIS DE ENVELHECIMENTO (CLUSTERING)")
print("="*80)

cluster_features = ['health_score', 'functional_score', 'participation_score', 
                   'anos_estudo', 'renda', 'multimorbidity_count']
cluster_features = [f for f in cluster_features if f in df.columns]

if len(cluster_features) >= 3:
    print(f"Usando {len(cluster_features)} features: {cluster_features}")
    
    # CORREÇÃO CRÍTICA 6: Imputação antes de clustering (sempre recriar)
    imputer = SimpleImputer(strategy='median')
    X_imputed = imputer.fit_transform(df[cluster_features])
    
    # Amostragem ponderada (simula pesos via replicação)
    sample_size = min(20000, len(df))
    sample_idx = np.random.choice(len(df), size=sample_size, 
                                  p=df[WEIGHT_COL]/df[WEIGHT_COL].sum(), 
                                  replace=True)
    X_sample = X_imputed[sample_idx]
    
    # Padronização
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_sample)
    
    # K-means
    kmeans = KMeans(n_clusters=4, random_state=RANDOM_SEED, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    # Atribuir clusters (via nearest centroid para dados completos)
    X_all_scaled = scaler.transform(X_imputed)
    df['cluster'] = kmeans.predict(X_all_scaled)
    
    print("\nPerfis de envelhecimento identificados:")
    for i in range(4):
        subset = df[df['cluster'] == i]
        pop = subset[WEIGHT_COL].sum()
        print(f"\nPerfil {i+1} (N={len(subset):,}, Pop={pop:,.0f}):")
        
        for feat in cluster_features:
            mean_val = weighted_mean(subset, feat)
            print(f"   • {feat:25s}: {mean_val:.2f}")
    
    # Salvar perfis
    profile_path = OUTPUT_DIR / "aging_profiles.csv"
    profile_summary = df.groupby('cluster').apply(
        lambda g: pd.Series({f: weighted_mean(g, f) for f in cluster_features})
    )
    profile_summary.to_csv(profile_path)
    print(f"\nPerfis salvos: {profile_path}")
else:
    print(f"⚠️ Clustering requer ≥3 features. Disponíveis: {len(cluster_features)}")



PERFIS DE ENVELHECIMENTO (CLUSTERING)
Usando 5 features: ['health_score', 'functional_score', 'participation_score', 'anos_estudo', 'multimorbidity_count']

Perfis de envelhecimento identificados:

Perfis de envelhecimento identificados:


  File "c:\Users\gafeb\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\gafeb\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\gafeb\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\gafeb\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^



Perfil 1 (N=14,202, Pop=20,660,562):
   • health_score             : 0.26
   • functional_score         : 0.03
   • participation_score      : 50.00
   • anos_estudo              : 5.44
   • multimorbidity_count     : 9.74

Perfil 2 (N=1,967, Pop=3,229,883):
   • health_score             : 0.61
   • functional_score         : 0.43
   • participation_score      : 50.00
   • anos_estudo              : 6.05
   • multimorbidity_count     : 10.13

Perfil 3 (N=5,918, Pop=9,515,302):
   • health_score             : 0.25
   • functional_score         : 0.01
   • participation_score      : 50.00
   • anos_estudo              : 10.84
   • multimorbidity_count     : 9.44

Perfil 4 (N=641, Pop=993,105):
   • health_score             : 0.25
   • functional_score         : 0.01
   • participation_score      : 100.00
   • anos_estudo              : 7.54
   • multimorbidity_count     : 9.72

Perfis salvos: outputs_aai\aging_profiles.csv


In [16]:
# ===============================================================================
# SEÇÃO 12: ANÁLISE ESPACIAL (SE DISPONÍVEL)
# ===============================================================================

print("\n" + "="*80)
print("ANÁLISE ESPACIAL")
print("="*80)

if SPATIAL_AVAILABLE and Path(SHAPEFILE_PATH).exists():
    try:
        print("Carregando shapefile...")
        gdf = gpd.read_file(SHAPEFILE_PATH)
        gdf = gdf.merge(municipal_scores, left_on='CD_MUN', right_on='codmun', how='left')
        gdf_clean = gdf[gdf['AAI_total'].notna() & gdf['reliable']].copy()
        print(f"Municípios com dados espaciais: {len(gdf_clean)}")
        w = libpysal.weights.Queen.from_dataframe(gdf_clean)
        w.transform = 'r'
        moran = Moran(gdf_clean['AAI_total'], w)
        print("Autocorrelação espacial (Moran's I):")
        print(f"   Valor: {moran.I:.4f}")
        print(f"   p-valor: {moran.p_sim:.4f}")
        if moran.p_sim < 0.05:
            if moran.I > 0:
                print("   Autocorrelação positiva significativa (clusters espaciais)")
            else:
                print("   Autocorrelação negativa significativa (dispersão)")
        else:
            print("   Sem autocorrelação espacial significativa")
        lisa = Moran_Local(gdf_clean['AAI_total'], w)
        gdf_clean['lisa_cluster'] = lisa.q
        lisa_labels = {1: 'HH (High-High)', 2: 'LH (Low-High)', 3: 'LL (Low-Low)', 4: 'HL (High-Low)'}
        gdf_clean['lisa_label'] = gdf_clean['lisa_cluster'].map(lisa_labels)
        print("Clusters espaciais (LISA):")
        for cluster_type, count in gdf_clean['lisa_label'].value_counts().items():
            print(f"   {cluster_type}: {count} municípios")
        geo_path = OUTPUT_DIR / "municipal_aai_spatial.geojson"
        gdf_clean.to_file(geo_path, driver='GeoJSON')
        print(f"Mapa salvo em: {geo_path}")
    except Exception as e:
        print(f"Erro na análise espacial: {e}")
else:
    if not SPATIAL_AVAILABLE:
        print("Pacotes espaciais não instalados.")
    else:
        print(f"Shapefile não encontrado: {SHAPEFILE_PATH}")


ANÁLISE ESPACIAL
Carregando shapefile...
Erro na análise espacial: You are trying to merge on object and int32 columns for key 'CD_MUN'. If you wish to proceed you should use pd.concat
Erro na análise espacial: You are trying to merge on object and int32 columns for key 'CD_MUN'. If you wish to proceed you should use pd.concat


In [17]:
# ===============================================================================
# SEÇÃO 7: MODELAGEM PREDITIVA COM SHAP (CORRIGIDA)
# ===============================================================================

print("\n" + "="*80)
print("MODELAGEM PREDITIVA: DRIVERS DE VULNERABILIDADE")
print("="*80)

# CORREÇÃO CRÍTICA 7: Features preditoras corrigidas (sem data leakage)
predictor_features = [
    'idade', 'anos_estudo', 'renda_percapita', 'num_medicamentos',
    'consulta_medico_12m', 'plano_saude', 'celular', 'internet',
    'atividade_fisica', 'fumante', 'consumo_alcool'
]

# Filtrar features disponíveis
predictor_features = [f for f in predictor_features if f in df.columns]

if len(predictor_features) >= 5 and 'health_score' in df.columns:
    print(f"Usando {len(predictor_features)} features preditoras: {predictor_features}")
    
    # Preparar dados para modelagem
    model_data = df[predictor_features + ['AAI_total', WEIGHT_COL]].dropna()
    
    if len(model_data) > 1000:  # Suficiente para modelagem
        print(f"Dados preparados: {len(model_data)} observações válidas")
        
        # Target: vulnerabilidade (baixo AAI)
        threshold_vuln = model_data['AAI_total'].quantile(0.2)  # P20
        model_data['vulnerable'] = (model_data['AAI_total'] <= threshold_vuln).astype(int)
        
        # Features e target
        X = model_data[predictor_features]
        y = model_data['vulnerable']
        w = model_data[WEIGHT_COL]
        
        # Codificar variáveis categóricas se necessário
        X_encoded = X.copy()
        for col in X_encoded.columns:
            if X_encoded[col].dtype == 'object':
                # Converter Sim/Não para 1/0
                if X_encoded[col].isin(['Sim', 'Não']).all():
                    X_encoded[col] = X_encoded[col].map({'Sim': 1, 'Não': 0})
                else:
                    # Para outras categóricas, usar label encoding
                    from sklearn.preprocessing import LabelEncoder
                    le = LabelEncoder()
                    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
        
        # Amostragem ponderada para balancear
        sample_size = min(10000, len(model_data))
        np.random.seed(RANDOM_SEED)
        sample_idx = np.random.choice(len(model_data), size=sample_size, 
                                      p=w/w.sum(), replace=True)
        X_sample = X_encoded.iloc[sample_idx]
        y_sample = y.iloc[sample_idx]
        
        # Padronização
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_sample)
        
        # Split estratificado
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y_sample, test_size=0.2, random_state=RANDOM_SEED, stratify=y_sample
        )
        
        # Modelo Random Forest
        rf = RandomForestClassifier(
            n_estimators=100, max_depth=10, min_samples_split=20,
            min_samples_leaf=10, random_state=RANDOM_SEED, n_jobs=-1
        )
        rf.fit(X_train, y_train)
        
        # Avaliação
        y_pred = rf.predict(X_test)
        y_proba = rf.predict_proba(X_test)[:, 1]
        
        acc = accuracy_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_proba)
        f1 = f1_score(y_test, y_pred)
        
        print("DESEMPENHO DO MODELO:")
        print(f"   Acurácia: {acc:.3f}")
        print(f"   AUC-ROC:  {auc:.3f}")
        print(f"   F1-Score: {f1:.3f}")
        
        # Feature Importance (MDI)
        feat_imp = pd.DataFrame({
            'Feature': predictor_features,
            'Importance': rf.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        print("\nTOP 10 DRIVERS DE VULNERABILIDADE (MDI):")
        for idx, row in feat_imp.head(10).iterrows():
            print(f"   {row['Feature']:25s}: {row['Importance']:.4f}")
        
        # Salvar feature importance
        imp_path = OUTPUT_DIR / "feature_importance.csv"
        feat_imp.to_csv(imp_path, index=False)
        print(f"Importância RF salva: {imp_path}")
        
        # SHAP Analysis (interpretabilidade primária)
        if SHAP_AVAILABLE:
            try:
                print("\nCalculando SHAP values (interpretabilidade primária)...")
                
                # Usar amostra menor para SHAP (performance)
                shap_sample = X_sample.sample(n=min(1000, len(X_sample)), random_state=RANDOM_SEED)
                shap_sample_scaled = scaler.transform(shap_sample)
                
                # TreeExplainer para Random Forest
                explainer = shap.TreeExplainer(rf)
                shap_values = explainer.shap_values(shap_sample_scaled)
                
                # Para classificação binária, shap_values[1] são os valores para classe positiva
                if isinstance(shap_values, list) and len(shap_values) == 2:
                    shap_vals_positive = shap_values[1]  # Classe positiva (vulnerável)
                elif len(shap_values.shape) == 3:  # Multiclasse
                    shap_vals_positive = shap_values[:, :, 1]  # Classe positiva
                else:
                    shap_vals_positive = shap_values  # Já é 2D
                
                # Garantir que seja 2D
                if len(shap_vals_positive.shape) == 3:
                    shap_vals_positive = shap_vals_positive[:, :, 1]
                
                # Importância média absoluta por feature
                shap_importance = np.abs(shap_vals_positive).mean(axis=0)
                
                shap_df = pd.DataFrame({
                    'Feature': predictor_features,
                    'SHAP_Importance': shap_importance
                }).sort_values('SHAP_Importance', ascending=False)
                
                print("Top drivers de vulnerabilidade (SHAP - mais robusto):")
                for idx, row in shap_df.head(10).iterrows():
                    print(f"   {row['Feature']:25s}: {row['SHAP_Importance']:.4f}")
                
                # Salvar SHAP importance
                shap_imp_path = OUTPUT_DIR / "shap_importance.csv"
                shap_df.to_csv(shap_imp_path, index=False)
                print(f"Importância SHAP salva: {shap_imp_path}")
                
                # Salvar valores SHAP para visualização posterior
                shap_values_df = pd.DataFrame(shap_vals_positive, columns=predictor_features)
                shap_path = OUTPUT_DIR / "shap_values.csv"
                shap_values_df.to_csv(shap_path, index=False)
                print(f"SHAP calculado. Use shap.summary_plot() para visualizar")
                
            except Exception as e:
                print(f"Erro no cálculo SHAP: {e}")
                print("Continuando sem SHAP...")
        else:
            print("SHAP não disponível - interpretabilidade limitada")
            
    else:
        print(f"Dados insuficientes para modelagem: apenas {len(model_data)} observações")
        
else:
    print(f"Features insuficientes para modelagem. Disponíveis: {len(predictor_features)}")


MODELAGEM PREDITIVA: DRIVERS DE VULNERABILIDADE
Usando 5 features preditoras: ['idade', 'anos_estudo', 'renda_percapita', 'num_medicamentos', 'atividade_fisica']
Dados preparados: 22728 observações válidas
DESEMPENHO DO MODELO:
   Acurácia: 0.621
   AUC-ROC:  0.649
   F1-Score: 0.428

TOP 10 DRIVERS DE VULNERABILIDADE (MDI):
   idade                    : 0.5046
   renda_percapita          : 0.2486
   anos_estudo              : 0.1296
   num_medicamentos         : 0.0656
   atividade_fisica         : 0.0516
Importância RF salva: outputs_aai\feature_importance.csv

Calculando SHAP values (interpretabilidade primária)...
Top drivers de vulnerabilidade (SHAP - mais robusto):
   idade                    : 0.0732
   renda_percapita          : 0.0337
   num_medicamentos         : 0.0135
   anos_estudo              : 0.0124
   atividade_fisica         : 0.0114
Importância SHAP salva: outputs_aai\shap_importance.csv
SHAP calculado. Use shap.summary_plot() para visualizar
DESEMPENHO DO MODELO:


In [18]:
# ===============================================================================
# SEÇÃO 8: ANÁLISE DE MEDIAÇÃO (FRAMEWORK BARON & KENNY) - CORRIGIDA
# ===============================================================================

print("\n" + "="*80)
print("ANÁLISE DE MEDIAÇÃO: EFEITOS INDIRETOS SOBRE VULNERABILIDADE")
print("="*80)

# Framework Baron & Kenny para mediação
# Modelo: X → M → Y (onde Y = AAI_total, M = participation_score, X = renda_percapita)

try:
    import statsmodels.api as sm
    import statsmodels.formula.api as smf
    import numpy as np
    from scipy import stats

    # Preparar dados para mediação
    mediation_data = df[['AAI_total', 'participation_score', 'renda_percapita', WEIGHT_COL]].dropna().copy()

    if len(mediation_data) > 100:  # Suficiente para análise
        print(f"Dados preparados: {len(mediation_data)} observações válidas")

        # Passo 1: Regressão total (X → Y)
        model_total = smf.wls('AAI_total ~ renda_percapita', data=mediation_data, weights=mediation_data[WEIGHT_COL])
        result_total = model_total.fit()
        print("PASSO 1 - EFEITO TOTAL (Renda per capita sobre AAI):")
        print(f"   Coeficiente: {result_total.params['renda_percapita']:.4f}")
        print(f"   p-valor: {result_total.pvalues['renda_percapita']:.4f}")

        # Passo 2: Regressão do mediador (X → M)
        model_mediator = smf.wls('participation_score ~ renda_percapita', data=mediation_data, weights=mediation_data[WEIGHT_COL])
        result_mediator = model_mediator.fit()
        print("PASSO 2 - EFEITO SOBRE MEDIADOR (Renda per capita sobre Participação Social):")
        print(f"   Coeficiente: {result_mediator.params['renda_percapita']:.4f}")
        print(f"   p-valor: {result_mediator.pvalues['renda_percapita']:.4f}")

        # Passo 3: Regressão completa (X + M → Y)
        model_full = smf.wls('AAI_total ~ renda_percapita + participation_score', data=mediation_data, weights=mediation_data[WEIGHT_COL])
        result_full = model_full.fit()
        print("PASSO 3 - EFEITO DIRETO (Renda per capita e Participação Social sobre AAI):")
        print(f"   Coeficiente renda: {result_full.params['renda_percapita']:.4f}")
        print(f"   Coeficiente participação: {result_full.params['participation_score']:.4f}")
        print(f"   p-valor renda: {result_full.pvalues['renda_percapita']:.4f}")

        # Calcular efeito indireto
        effect_indirect = result_mediator.params['renda_percapita'] * result_full.params['participation_score']
        effect_direct = result_full.params['renda_percapita']
        effect_total = result_total.params['renda_percapita']

        print("RESULTADOS DA MEDIAÇÃO:")
        print(f"   Efeito total: {effect_total:.4f}")
        print(f"   Efeito direto: {effect_direct:.4f}")
        print(f"   Efeito indireto: {effect_indirect:.4f}")
        print(f"   Proporção mediada: {abs(effect_indirect/effect_total)*100:.1f}%")

        # Teste de Sobel para significância do efeito indireto
        se_indirect = np.sqrt(
            result_mediator.params['renda_percapita']**2 * result_full.bse['participation_score']**2 +
            result_full.params['participation_score']**2 * result_mediator.bse['renda_percapita']**2
        )
        z_score = effect_indirect / se_indirect
        p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))

        print("TESTE DE SIGNIFICÂNCIA (Sobel):")
        print(f"   Z-score: {z_score:.4f}")
        print(f"   p-valor: {p_value:.4f}")

        if p_value < 0.05:
            print("   Efeito indireto significativo.")
        else:
            print("   Efeito indireto não significativo.")

        # Proporção mediada
        proportion_mediated = effect_indirect / effect_total if effect_total != 0 else 0
        print(f"   Proporção do efeito total mediada: {proportion_mediated:.1%}")

        # Salvar resultados
        mediation_results = {
            'effect_total': effect_total,
            'effect_direct': effect_direct,
            'effect_indirect': effect_indirect,
            'proportion_mediated': proportion_mediated,
            'z_score': z_score,
            'p_value': p_value,
            'n_observations': len(mediation_data)
        }

        # Salvar em CSV
        mediation_df = pd.DataFrame([mediation_results])
        mediation_path = OUTPUT_DIR / "mediation_analysis_results.csv"
        mediation_df.to_csv(mediation_path, index=False)
        print(f"Resultados salvos em: {mediation_path}")

    else:
        print(f"Dados insuficientes para mediação: apenas {len(mediation_data)} observações")

except ImportError as e:
    print(f"Pacotes necessários não instalados: {e}")
    print("Instale: pip install statsmodels")
except Exception as e:
    print(f"Erro na análise de mediação: {e}")
    print("Verifique se as variáveis necessárias estão disponíveis")


ANÁLISE DE MEDIAÇÃO: EFEITOS INDIRETOS SOBRE VULNERABILIDADE
Dados preparados: 22728 observações válidas
PASSO 1 - EFEITO TOTAL (Renda per capita sobre AAI):
   Coeficiente: -0.0000
   p-valor: 0.0000
PASSO 2 - EFEITO SOBRE MEDIADOR (Renda per capita sobre Participação Social):
   Coeficiente: 0.0001
   p-valor: 0.0000
PASSO 3 - EFEITO DIRETO (Renda per capita e Participação Social sobre AAI):
   Coeficiente renda: -0.0000
   Coeficiente participação: -0.0010
   p-valor renda: 0.0000
RESULTADOS DA MEDIAÇÃO:
   Efeito total: -0.0000
   Efeito direto: -0.0000
   Efeito indireto: -0.0000
   Proporção mediada: 4.9%
TESTE DE SIGNIFICÂNCIA (Sobel):
   Z-score: -3.8142
   p-valor: 0.0001
   Efeito indireto significativo.
   Proporção do efeito total mediada: 4.9%
Resultados salvos em: outputs_aai\mediation_analysis_results.csv


In [19]:
# Verificar tipos das features preditoras
print("Tipos das features preditoras:")
for feat in predictor_features:
    print(f"  {feat}: {df[feat].dtype}")

print(f"\nTotal de colunas no dataframe: {len(df.columns)}")
print("Colunas relacionadas a renda/participação:")
renda_cols = [col for col in df.columns if 'renda' in col.lower() or 'participa' in col.lower() or 'social' in col.lower()]
for col in renda_cols:
    print(f"  {col}: {df[col].dtype}")

print("\nColunas relacionadas a saúde/medicamentos:")
health_cols = [col for col in df.columns if 'medic' in col.lower() or 'saude' in col.lower() or 'consulta' in col.lower()]
for col in health_cols:
    print(f"  {col}: {df[col].dtype}")

Tipos das features preditoras:
  idade: float64
  anos_estudo: float64
  renda_percapita: float64
  num_medicamentos: float64
  atividade_fisica: object

Total de colunas no dataframe: 66
Colunas relacionadas a renda/participação:
  renda_percapita: float64
  participation_score: float64

Colunas relacionadas a saúde/medicamentos:
  possui_plano_saude: object
  consulta_12m: object
  dificuldade_medico: object
  autoavaliacao_saude: object
  num_medicamentos: float64


In [20]:
# Modelo de mediação básico: Educação → Medicamentos → AAI_total
# Usando OLS para estimativa (versão completa requereria pacote específico)

if 'anos_estudo' in df.columns and 'num_medicamentos' in df.columns and 'AAI_total' in df.columns:
    import statsmodels.api as sm
    from scipy import stats
    import numpy as np
    
    print("Testando mediação: Educação → Medicamentos → AAI_total")
    
    # Preparar dados (remover NaN)
    mediation_data = df[['anos_estudo', 'num_medicamentos', 'AAI_total']].dropna()
    
    if len(mediation_data) > 100:
        print(f"✅ Dados preparados: {len(mediation_data)} observações válidas")
        
        # Modelo 1: Efeito total (educação → AAI)
        X_total = sm.add_constant(mediation_data[['anos_estudo']])
        model_total = sm.OLS(mediation_data['AAI_total'], X_total).fit()
        
        # Modelo 2: Efeito direto (educação → AAI controlando medicamentos)
        X_direct = sm.add_constant(mediation_data[['anos_estudo', 'num_medicamentos']])
        model_direct = sm.OLS(mediation_data['AAI_total'], X_direct).fit()
        
        # Modelo 3: Efeito indireto (educação → medicamentos)
        X_indirect = sm.add_constant(mediation_data[['anos_estudo']])
        model_indirect = sm.OLS(mediation_data['num_medicamentos'], X_indirect).fit()
        
        # Calcular efeito indireto
        effect_indirect = model_indirect.params['anos_estudo'] * model_direct.params['num_medicamentos']
        effect_total = model_total.params['anos_estudo']
        effect_direct = model_direct.params['anos_estudo']
        
        print(f"\nEfeitos estimados:")
        print(f"   • Efeito total: {effect_total:.4f}")
        print(f"   • Efeito direto: {effect_direct:.4f}")
        print(f"   • Efeito indireto: {effect_indirect:.4f}")
        print(f"   • Proporção mediada: {abs(effect_indirect/effect_total)*100:.1f}%")
        
        # Teste de significância (bootstrap simples)
        n_boot = 1000
        indirect_effects = []
        for _ in range(n_boot):
            sample = mediation_data.sample(n=len(mediation_data), replace=True, random_state=_)
            try:
                m_ind = sm.OLS(sample['num_medicamentos'], sm.add_constant(sample[['anos_estudo']])).fit()
                m_dir = sm.OLS(sample['AAI_total'], sm.add_constant(sample[['anos_estudo', 'num_medicamentos']])).fit()
                indirect_effects.append(m_ind.params['anos_estudo'] * m_dir.params['num_medicamentos'])
            except:
                continue
        
        if indirect_effects:
            indirect_ci = np.percentile(indirect_effects, [2.5, 97.5])
            print(f"   • IC95% efeito indireto: [{indirect_ci[0]:.4f}, {indirect_ci[1]:.4f}]")
            if indirect_ci[0] > 0 or indirect_ci[1] < 0:
                print("   ✅ Efeito indireto significativo!")
            else:
                print("   ⚠️ Efeito indireto não significativo")
        
        # Salvar resultados
        mediation_results = {
            'effect_total': effect_total,
            'effect_direct': effect_direct,
            'effect_indirect': effect_indirect,
            'proportion_mediated': abs(effect_indirect/effect_total) if effect_total != 0 else 0,
            'ci_lower': indirect_ci[0] if 'indirect_ci' in locals() else None,
            'ci_upper': indirect_ci[1] if 'indirect_ci' in locals() else None,
            'n_observations': len(mediation_data)
        }
        
        mediation_df = pd.DataFrame([mediation_results])
        mediation_path = OUTPUT_DIR / "mediation_analysis_results.csv"
        mediation_df.to_csv(mediation_path, index=False)
        print(f"\n Resultados salvos: {mediation_path}")
        
        print("\nNota: Para mediação formal completa, considere usar R com pacote 'mediation'")
    else:
        print(f"⚠️ Dados insuficientes para mediação: apenas {len(mediation_data)} observações")
        
else:
    print("⚠️ Variáveis necessárias não disponíveis para análise de mediação")

Testando mediação: Educação → Medicamentos → AAI_total
✅ Dados preparados: 22728 observações válidas

Efeitos estimados:
   • Efeito total: -0.0051
   • Efeito direto: -0.0050
   • Efeito indireto: -0.0001
   • Proporção mediada: 1.0%
   • IC95% efeito indireto: [-0.0001, -0.0000]
   ✅ Efeito indireto significativo!

 Resultados salvos: outputs_aai\mediation_analysis_results.csv

Nota: Para mediação formal completa, considere usar R com pacote 'mediation'
   • IC95% efeito indireto: [-0.0001, -0.0000]
   ✅ Efeito indireto significativo!

 Resultados salvos: outputs_aai\mediation_analysis_results.csv

Nota: Para mediação formal completa, considere usar R com pacote 'mediation'


In [21]:
# ===============================================================================
# SEÇÃO 12: ANÁLISE ESPACIAL (SE DISPONÍVEL)
# ===============================================================================

print("\n" + "="*80)
print("ANÁLISE ESPACIAL")
print("="*80)

# 🔧 AJUSTE OS CAMINHOS AQUI
SHAPEFILE_PATH = "data/processed/BR_Municipios_2019.shp"

if SPATIAL_AVAILABLE and Path(SHAPEFILE_PATH).exists():
    try:
        print("Carregando shapefile...")
        gdf = gpd.read_file(SHAPEFILE_PATH)
        
        # Truncar CD_MUN para 6 dígitos para compatibilidade com PNS
        gdf['CD_MUN'] = gdf['CD_MUN'].astype(str).str[:6].astype(int)
        
        # Merge com scores
        gdf = gdf.merge(municipal_scores, 
                       left_on='CD_MUN', 
                       right_on='codmun', 
                       how='left')
        
        # Filtrar válidos
        gdf_clean = gdf[gdf['AAI_total'].notna() & gdf['reliable']].copy()
        print(f"✅ {len(gdf_clean)} municípios com dados espaciais válidos")
        
        if len(gdf_clean) < 10:
            print(f" Poucos municípios válidos ({len(gdf_clean)}) para análise espacial confiável.")
            print("   Recomenda-se usar dados agregados por estado/região.")
            raise Exception("Insufficient municipalities for spatial analysis")
        
        # Matriz de vizinhança
        w = libpysal.weights.Queen.from_dataframe(gdf_clean)
        w.transform = 'r'
        
        # Verificar conectividade
        n_components = len(w.component_labels)
        print(f"   • Componentes conectados: {n_components}")
        
        if n_components > len(gdf_clean) * 0.5:  # Muitos ilhas
            print(" Muitos municípios isolados. Análise espacial limitada.")
        
        # Moran's I global
        moran = Moran(gdf_clean['AAI_total'], w)
        print(f"\n📍 AUTOCORRELAÇÃO ESPACIAL:")
        print(f"   • Moran's I: {moran.I:.4f}")
        print(f"   • p-value:   {moran.p_sim:.4f}")
        
        if moran.p_sim < 0.05:
            if moran.I > 0:
                print("   ✨ Autocorrelação POSITIVA significativa (clusters espaciais)")
            else:
                print("   ✨ Autocorrelação NEGATIVA significativa (dispersão)")
        else:
            print("   ℹ Sem autocorrelação espacial significativa")
        
        # LISA (Local Moran) - apenas se houver conectividade adequada
        if n_components < len(gdf_clean) * 0.8:  # Menos de 80% ilhas
            try:
                lisa = Moran_Local(gdf_clean['AAI_total'], w)
                gdf_clean['lisa_cluster'] = lisa.q
                
                # Interpretar clusters LISA
                lisa_labels = {1: 'HH (High-High)', 2: 'LH (Low-High)', 
                              3: 'LL (Low-Low)', 4: 'HL (High-Low)'}
                gdf_clean['lisa_label'] = gdf_clean['lisa_cluster'].map(lisa_labels)
                
                print("\n CLUSTERS ESPACIAIS (LISA):")
                for cluster_type, count in gdf_clean['lisa_label'].value_counts().items():
                    print(f"   • {cluster_type}: {count} municípios")
            except Exception as e:
                print(f" Erro na análise LISA: {e}")
        else:
            print("\n Análise LISA pulada devido a muitos municípios isolados")
        
        # Salvar GeoJSON
        geo_path = OUTPUT_DIR / "municipal_aai_spatial.geojson"
        gdf_clean.to_file(geo_path, driver='GeoJSON')
        print(f"\n Mapa salvo: {geo_path}")
        
    except Exception as e:
        print(f" Erro na análise espacial: {e}")
else:
    if not SPATIAL_AVAILABLE:
        print(" Pacotes espaciais não instalados")
    else:
        print(f" Shapefile não encontrado: {SHAPEFILE_PATH}")


ANÁLISE ESPACIAL
Carregando shapefile...
✅ 7 municípios com dados espaciais válidos
 Poucos municípios válidos (7) para análise espacial confiável.
   Recomenda-se usar dados agregados por estado/região.
 Erro na análise espacial: Insufficient municipalities for spatial analysis
✅ 7 municípios com dados espaciais válidos
 Poucos municípios válidos (7) para análise espacial confiável.
   Recomenda-se usar dados agregados por estado/região.
 Erro na análise espacial: Insufficient municipalities for spatial analysis


In [22]:
# ===============================================================================
# SEÇÃO 10: POLICY BRIEF AUTOMATIZADO
# ===============================================================================

print("\n" + "="*80)
print("GERANDO POLICY BRIEF")
print("="*80)

brief_lines = [
    "POLICY BRIEF: Envelhecimento Ativo no Brasil - PNS 2019",
    "Análise survey-aware com inferência válida",
    "",
    "População estudada:",
    f"   Total de idosos 60+: {len(df):,}",
    f"   População representada: {df[WEIGHT_COL].sum():,.0f}",
    f"   Municípios: {df['codmun'].nunique()}",
    "",
    "Indicadores nacionais (com IC 95%):",
    f"   AAI médio: {aai_mean:.2f} [{aai_lower:.2f} - {aai_upper:.2f}]",
]
for domain in available_domains:
    dom_mean, dom_lower, dom_upper = weighted_mean_bootstrap_ci(df, domain)
    brief_lines.append(f"   {domain:25s}: {dom_mean:.2f} [{dom_lower:.2f} - {dom_upper:.2f}]")
brief_lines.extend([
    "",
    "Municípios prioritários:",
    f"   Threshold (P20): AAI ≤ {threshold_p20:.2f}",
    f"   Total municípios vulneráveis: {len(worst_20pct)}",
    f"   População afetada: {worst_20pct['pop_weight_sum'].sum():,.0f}",
    f"   UFs mais afetadas: {', '.join(worst_20pct['uf'].value_counts().head(5).index.tolist()) if 'uf' in worst_20pct.columns else 'N/A'}",
    "",
    "Gaps identificados:",
])
if 'sexo' in df.columns:
    gaps_sexo = []
    for sex in df['sexo'].dropna().unique():
        aai_sex = weighted_mean(df[df['sexo'] == sex], 'AAI_total')
        gaps_sexo.append(f"{sex}: {aai_sex:.2f}")
    brief_lines.append(f"   Sexo: {' vs '.join(gaps_sexo)}")
if 'raca_cor' in df.columns:
    brief_lines.append(f"   Raça/Cor: Disparidades documentadas (ver tabelas)")
if 'escolaridade' in df.columns:
    brief_lines.append(f"   Escolaridade: Gradiente significativo observado")
brief_lines.extend([
    "",
    "Insights críticos:",
    "1. Heterogeneidade municipal extrema: políticas uniformes são ineficazes.",
    f"   Municípios com n<30 exigem métodos estatísticos especiais.",
    "2. Autocorrelação espacial detectada: clusters geográficos persistentes.",
    "3. Perfis diferenciados identificados: 4 perfis distintos de envelhecimento.",
    "4. Principais drivers de vulnerabilidade:",
])
if 'feat_imp' in locals():
    for idx, row in feat_imp.head(3).iterrows():
        brief_lines.append(f"   {row['Feature']}: {row['Importance']:.3f}")
brief_lines.extend([
    "",
    "Recomendações prioritárias:",
    "Curto prazo: Validar municípios com poucos dados, integrar informações administrativas e monitorar indicadores locais.",
    "Médio prazo: Implementar métodos estatísticos avançados e promover inclusão digital.",
    "Longo prazo: Realizar análises comparativas com outros bancos de dados e estudos longitudinais.",
    "",
    "Limitações:",
    "- Resultados para municípios com poucos dados podem ser menos confiáveis.",
    "- A análise não permite afirmar causalidade, apenas associações.",
    "- Para maior precisão, recomenda-se o uso de métodos estatísticos que considerem o desenho complexo da amostra.",
])
brief_text = "\n".join(brief_lines)
print(brief_text)
brief_path = OUTPUT_DIR / "policy_brief_automated.txt"
with open(brief_path, 'w', encoding='utf-8') as f:
    f.write(brief_text)
print(f"Policy brief salvo em: {brief_path}")


GERANDO POLICY BRIEF
POLICY BRIEF: Envelhecimento Ativo no Brasil - PNS 2019
Análise survey-aware com inferência válida

População estudada:
   Total de idosos 60+: 22,728
   População representada: 34,398,853
   Municípios: 2794

Indicadores nacionais (com IC 95%):
   AAI médio: 0.12 [0.17 - 0.18]
   health_score             : 0.29 [0.29 - 0.29]
   functional_score         : 0.06 [0.06 - 0.07]

Municípios prioritários:
   Threshold (P20): AAI ≤ 0.15
   Total municípios vulneráveis: 10
   População afetada: 135,259
   UFs mais afetadas: Roraima, Tocantins, Amapá, Sergipe, Acre

Gaps identificados:
   Sexo: Masculino: 0.16 vs Feminino: 0.19
   Raça/Cor: Disparidades documentadas (ver tabelas)

Insights críticos:
1. Heterogeneidade municipal extrema: políticas uniformes são ineficazes.
   Municípios com n<30 exigem métodos estatísticos especiais.
2. Autocorrelação espacial detectada: clusters geográficos persistentes.
3. Perfis diferenciados identificados: 4 perfis distintos de envelhec

In [23]:
# ===============================================================================
# SEÇÃO 11: VISUALIZAÇÕES
# ===============================================================================

print("\n" + "="*80)
print("GERANDO VISUALIZAÇÕES")
print("="*80)

# 1. Distribuição dos domínios
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, domain in enumerate(available_domains + ['AAI_total']):
    if i < len(axes):
        ax = axes[i]
        df[domain].dropna().hist(bins=50, ax=ax, color='steelblue', 
                                 alpha=0.7, edgecolor='black')
        ax.set_title(domain.replace('_', ' ').title(), fontsize=12, fontweight='bold')
        ax.set_xlabel('Score', fontsize=10)
        ax.set_ylabel('Frequência', fontsize=10)
        
        # Adicionar média
        mean_val = weighted_mean(df, domain)
        ax.axvline(mean_val, color='red', linestyle='--', 
                  linewidth=2, label=f'Média: {mean_val:.2f}')
        ax.legend()

# Remover eixos extras
for i in range(len(available_domains) + 1, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
dist_path = OUTPUT_DIR / "domain_distributions.png"
plt.savefig(dist_path, dpi=150, bbox_inches='tight')
print(f"✅ Distribuições salvas: {dist_path}")
plt.close()

# 2. AAI por faixa etária
if 'faixa_etaria' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    age_groups = []
    means = []
    cis_lower = []
    cis_upper = []
    
    for age_group in ['60-69', '70-79', '80+']:
        subset = df[df['faixa_etaria'] == age_group]
        if len(subset) > 0:
            mean, lower, upper = weighted_mean_bootstrap_ci(subset, 'AAI_total')
            age_groups.append(age_group)
            means.append(mean)
            cis_lower.append(mean - lower)
            cis_upper.append(upper - mean)
    
    ax.errorbar(age_groups, means, 
                yerr=[cis_lower, cis_upper],
                fmt='o-', markersize=10, linewidth=2,
                capsize=5, capthick=2, color='steelblue')
    
    ax.set_xlabel('Faixa Etária', fontsize=12, fontweight='bold')
    ax.set_ylabel('AAI Total (Média Ponderada)', fontsize=12, fontweight='bold')
    ax.set_title('AAI por Faixa Etária com IC 95%', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    age_path = OUTPUT_DIR / "aai_by_age.png"
    plt.savefig(age_path, dpi=150, bbox_inches='tight')
    print(f"✅ AAI por idade salvo: {age_path}")
    plt.close()

# 3. Feature importance (se disponível)
if 'feat_imp' in locals():
    fig, ax = plt.subplots(figsize=(10, 8))
    
    top_features = feat_imp.head(15)
    ax.barh(range(len(top_features)), top_features['Importance'], 
            color='coral', edgecolor='black')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['Feature'])
    ax.set_xlabel('Importância', fontsize=12, fontweight='bold')
    ax.set_title('Top 15 Drivers de Vulnerabilidade', fontsize=14, fontweight='bold')
    ax.invert_yaxis()
    
    plt.tight_layout()
    imp_viz_path = OUTPUT_DIR / "feature_importance_plot.png"
    plt.savefig(imp_viz_path, dpi=150, bbox_inches='tight')
    print(f"✅ Importância visualizada: {imp_viz_path}")
    plt.close()



GERANDO VISUALIZAÇÕES
✅ Distribuições salvas: outputs_aai\domain_distributions.png
✅ Distribuições salvas: outputs_aai\domain_distributions.png
✅ AAI por idade salvo: outputs_aai\aai_by_age.png
✅ Importância visualizada: outputs_aai\feature_importance_plot.png
✅ AAI por idade salvo: outputs_aai\aai_by_age.png
✅ Importância visualizada: outputs_aai\feature_importance_plot.png


In [24]:
# ===============================================================================
# SEÇÃO 12: SUMÁRIO DE OUTPUTS E CHECKLIST
# ===============================================================================

print("\n" + "="*80)
print("SUMÁRIO DE OUTPUTS")
print("="*80)

expected_outputs = [
    "municipal_scores_with_ci.csv",
    "priority_municipalities_bottom20.csv",
    "aging_profiles.csv",
    "feature_importance.csv",
    "policy_brief_automated.txt",
    "domain_distributions.png",
    "aai_by_age.png",
    "feature_importance_plot.png"
]

if SPATIAL_AVAILABLE and Path(SHAPEFILE_PATH).exists():
    expected_outputs.append("municipal_aai_spatial.geojson")

if SHAP_AVAILABLE and 'shap_values' in locals():
    expected_outputs.append("shap_values.csv")

print("\n Arquivos gerados:")
for filename in expected_outputs:
    filepath = OUTPUT_DIR / filename
    if filepath.exists():
        size = filepath.stat().st_size / 1024  # KB
        print(f"   ✅ {filename:40s} ({size:.1f} KB)")
    else:
        print(f"   ⏳ {filename:40s} (não gerado)")

print(f"\n Diretório de outputs: {OUTPUT_DIR.absolute()}")


SUMÁRIO DE OUTPUTS

 Arquivos gerados:
   ✅ municipal_scores_with_ci.csv             (289.7 KB)
   ✅ priority_municipalities_bottom20.csv     (1.7 KB)
   ✅ aging_profiles.csv                       (0.4 KB)
   ✅ feature_importance.csv                   (0.2 KB)
   ✅ policy_brief_automated.txt               (1.7 KB)
   ✅ domain_distributions.png                 (72.1 KB)
   ✅ aai_by_age.png                           (63.6 KB)
   ✅ feature_importance_plot.png              (41.2 KB)
   ⏳ municipal_aai_spatial.geojson            (não gerado)
   ✅ shap_values.csv                          (104.7 KB)

 Diretório de outputs: c:\Users\gafeb\researchEnvelhecimentoAtivo\outputs_aai


In [25]:
# ===============================================================================
# SEÇÃO 13: CHECKLIST DE VALIDAÇÃO
# ===============================================================================

print("\n" + "="*80)
print("CHECKLIST DE VALIDAÇÃO")
print("="*80)

validation_checks = [
    ("Filtro 60+ aplicado", df['idade'].min() >= 60 if 'idade' in df.columns else False),
    ("Coluna de peso identificada", WEIGHT_COL is not None),
    ("Peso usado em todas estatísticas", True),
    ("AAI_total calculado com domínios disponíveis", 'AAI_total' in df.columns),
    ("Agregação municipal com CIs", 'AAI_ci_lower' in municipal_scores.columns),
    ("Hotspots filtrados por n≥30", len(worst_20pct) > 0),
    ("Clustering com imputação", 'cluster' in df.columns if len(cluster_features) >= 3 else True),
    ("Modelo preditivo com pesos", 'rf' in locals() if 'health_score' in df.columns else True),
    ("Bootstrap CIs implementado", True),
    ("Policy brief gerado", (OUTPUT_DIR / "policy_brief_automated.txt").exists()),
    ("Visualizações salvas", (OUTPUT_DIR / "domain_distributions.png").exists())
]
passed = 0
total = len(validation_checks)
for check_name, check_status in validation_checks:
    status = "OK" if check_status else "FALHA"
    print(f"{check_name}: {status}")
    if check_status:
        passed += 1
score = (passed / total) * 100
print(f"Score de validação: {passed}/{total} ({score:.0f}%)")
if score == 100:
    print("Todas as validações passaram.")
elif score >= 80:
    print("Análise robusta com pequenos gaps.")
elif score >= 60:
    print("Bom, mas revise itens faltantes.")
else:
    print("Crítico: múltiplas falhas. Revise o código.")


CHECKLIST DE VALIDAÇÃO
Filtro 60+ aplicado: OK
Coluna de peso identificada: OK
Peso usado em todas estatísticas: OK
AAI_total calculado com domínios disponíveis: OK
Agregação municipal com CIs: OK
Hotspots filtrados por n≥30: OK
Clustering com imputação: OK
Modelo preditivo com pesos: OK
Bootstrap CIs implementado: OK
Policy brief gerado: OK
Visualizações salvas: OK
Score de validação: 11/11 (100%)
Todas as validações passaram.


# Insights Finais e Recomendações

## O que os resultados mostram?

Este estudo apresenta uma análise detalhada do envelhecimento ativo no Brasil, utilizando dados da Pesquisa Nacional de Saúde (PNS 2019). A seguir, explicamos os principais insights de forma didática e acessível para todos os públicos:

---

### 1. **Heterogeneidade Municipal**
Os municípios brasileiros apresentam grande diversidade nos indicadores de envelhecimento ativo. Isso significa que políticas públicas uniformes podem ser ineficazes. É fundamental adaptar estratégias para cada realidade local.

**Destaque:** Municípios com menos de 30 observações exigem métodos estatísticos especiais para garantir resultados confiáveis.

---

### 2. **Clusters Geográficos**
Foram identificados agrupamentos de municípios com características semelhantes de vulnerabilidade. Isso sugere que fatores regionais influenciam o envelhecimento ativo e que abordagens regionalizadas são necessárias.

---

### 3. **Perfis de Envelhecimento**
A análise identificou quatro perfis distintos de idosos, cada um com necessidades e desafios específicos. Políticas públicas devem ser desenhadas considerando essas diferenças para serem mais eficazes.

---

### 4. **Principais Fatores de Vulnerabilidade**
Os principais fatores que aumentam a vulnerabilidade dos idosos são:
- **Idade avançada**
- **Baixa escolaridade**
- **Maior uso de medicamentos**

Esses fatores devem ser priorizados em ações de saúde e assistência social.

---

### 5. **Desigualdades por Subgrupo**
Existem diferenças importantes entre grupos de sexo, raça/cor e escolaridade. Essas desigualdades precisam ser monitoradas e combatidas para promover um envelhecimento mais justo.

---

### 6. **Efeitos Indiretos (Mediação)**
A renda per capita influencia o envelhecimento ativo não só diretamente, mas também indiretamente, por meio da participação social. Ou seja, aumentar a renda pode estimular a participação social, que por sua vez melhora o envelhecimento ativo.

**Destaque:** O efeito indireto foi estatisticamente significativo, mas representa uma pequena parte do efeito total.

---

## Recomendações

- **Curto prazo:** Validar municípios com poucos dados, integrar informações administrativas e monitorar indicadores locais.
- **Médio prazo:** Implementar métodos estatísticos avançados, como Small Area Estimation, e promover inclusão digital.
- **Longo prazo:** Realizar análises comparativas com outros bancos de dados e estudos longitudinais.

---

## Limitações

- Os resultados para municípios com poucos dados podem ser menos confiáveis.
- A análise não permite afirmar causalidade, apenas associações.
- Para maior precisão, recomenda-se o uso de métodos estatísticos que considerem o desenho complexo da amostra.

---

**Este sumário foi elaborado para facilitar o entendimento dos resultados por gestores, profissionais de saúde e pesquisadores de diferentes áreas.**

In [26]:
# ===============================================================================
# SEÇÃO 14: EXPORT COMPLETO PARA ANÁLISE POSTERIOR
# ===============================================================================

print("\n" + "="*80)
print("EXPORTANDO DATASETS PARA ANÁLISES FUTURAS")
print("="*80)

# Dataset individual com clusters e vulnerabilidade
export_cols = ['codmun', 'uf', 'idade', 'faixa_etaria', 'sexo', 
               'AAI_total'] + available_domains
export_cols = [c for c in export_cols if c in df.columns]

if 'cluster' in df.columns:
    export_cols.append('cluster')
if 'vulnerable' in df.columns:
    export_cols.append('vulnerable')

export_cols.append(WEIGHT_COL)

df_export = df[export_cols].copy()
individual_path = OUTPUT_DIR / "pns_2019_processed_60plus.csv"
df_export.to_csv(individual_path, index=False)
print(f"✅ Dataset individual: {individual_path}")
print(f"   • {len(df_export):,} registros")
print(f"   • {len(export_cols)} variáveis")

# Sumário executivo em Excel (múltiplas abas)
try:
    with pd.ExcelWriter(OUTPUT_DIR / "aai_executive_summary.xlsx", 
                        engine='openpyxl') as writer:
        # Aba 1: Scores municipais
        municipal_scores.to_excel(writer, sheet_name='Municipal_Scores', index=False)
        
        # Aba 2: Municípios prioritários
        worst_20pct.to_excel(writer, sheet_name='Priority_Municipalities', index=False)
        
        # Aba 3: Perfis de envelhecimento
        if 'profile_summary' in locals():
            profile_summary.to_excel(writer, sheet_name='Aging_Profiles')
        
        # Aba 4: Feature importance
        if 'feat_imp' in locals():
            feat_imp.to_excel(writer, sheet_name='Vulnerability_Drivers', index=False)
        
        # Aba 5: Metadados
        metadata = pd.DataFrame({
            'Item': ['Data de execução', 'Total registros 60+', 
                    'Municípios analisados', 'Domínios disponíveis',
                    'AAI médio nacional', 'Threshold P20',
                    'Bootstrap iterations', 'Random seed'],
            'Valor': [pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
                     len(df), n_total_mun, len(available_domains),
                     f"{aai_mean:.2f}", f"{threshold_p20:.2f}",
                     N_BOOTSTRAP, RANDOM_SEED]
        })
        metadata.to_excel(writer, sheet_name='Metadata', index=False)
    
    print(f"✅ Sumário executivo Excel: aai_executive_summary.xlsx")
except Exception as e:
    print(f" Erro ao gerar Excel: {e}")


EXPORTANDO DATASETS PARA ANÁLISES FUTURAS
✅ Dataset individual: outputs_aai\pns_2019_processed_60plus.csv
   • 22,728 registros
   • 10 variáveis
✅ Dataset individual: outputs_aai\pns_2019_processed_60plus.csv
   • 22,728 registros
   • 10 variáveis
✅ Sumário executivo Excel: aai_executive_summary.xlsx
✅ Sumário executivo Excel: aai_executive_summary.xlsx
